In [28]:
import pandas as pd
import warnings
import requests, zipfile, io, os
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')

In [29]:
# download file from url, extract it and store it in the dest_folder
def download_and_extract(url, dest_folder):
  r = requests.get(url)
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall(dest_folder)

In [30]:
# define folders and urls
healthy_path = './healthy'
healthy_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewHealthy/Signal.zip'
parkinson_path = './parkinson'
parkinson_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewPatients/Signal.zip'

In [31]:
download_and_extract(healthy_url, healthy_path)

In [32]:
download_and_extract(parkinson_url, parkinson_path)

In [33]:
# actual downloaded signals
healthy_signals = f'{healthy_path}/Signal/'
parkinson_signals = f'{parkinson_path}/Signal/'

In [34]:
# define Input and labels (0 = healthy, parkinson = 1)
X = []
y = []
seq = 500

In [35]:
# create dataset with sequence length 500

# healthy signals
for filename in os.listdir(healthy_signals):
  if filename.endswith('txt'):
    df = pd.read_csv(healthy_signals + filename, sep='\s+', header=None, comment='#')
    count = len(df.index) // seq
    for i in range(count):
      X.append((df.iloc[i * seq : (i + 1) * seq]).to_numpy())
      y.append(0)

# parkinson signals
for filename in os.listdir(parkinson_signals):
  if filename.endswith('txt'):
    df = pd.read_csv(parkinson_signals + filename, sep='\s+', header=None, comment='#')
    count = len(df.index) // seq
    for i in range(count):
      X.append((df.iloc[i * seq : (i + 1) * seq]).to_numpy())
      y.append(1) 


In [36]:
len(X)

29088

In [37]:
# convert lists to np arrays
X = np.array(X)
y = np.array(y)

In [38]:
X.shape

(29088, 500, 6)

In [39]:
y.shape

(29088,)

In [40]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [41]:
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size=0.2)

In [42]:
X_valid.shape

(5818, 500, 6)

In [43]:
model = keras.models.Sequential([
keras.layers.LSTM(128, return_sequences=True,  input_shape=(X_train.shape[1], 6)),
keras.layers.BatchNormalization(),
keras.layers.LSTM(128),
keras.layers.BatchNormalization(),
keras.layers.Dense(1, activation="sigmoid")
])


In [44]:
model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])


In [45]:
model.fit(X_train, y_train, epochs = 120, batch_size = 32, validation_data=(X_valid, y_valid))

In [46]:
model.evaluate(X_test, y_test)


146/146 [==============================] - 11s 75ms/step - loss: 0.2391 - accuracy: 0.6682


[0.23906560242176056, 0.6682423949241638]

In [ ]:
model.evaluate(X_valid, y_valid)